In [1]:
# %load lgbm_olivier2.py
import re
import string
import os
import psutil
import gc

import numpy as np
import pandas as pd
import lightgbm as lgb

from collections import defaultdict

from scipy.sparse import hstack
from scipy.sparse import csr_matrix

from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score


In [2]:

cont_patterns = [
        (b'US', b'United States'),
        (b'IT', b'Information Technology'),
        (b'(W|w)on\'t', b'will not'),
        (b'(C|c)an\'t', b'can not'),
        (b'(I|i)\'m', b'i am'),
        (b'(A|a)in\'t', b'is not'),
        (b'(\w+)\'ll', b'\g<1> will'),
        (b'(\w+)n\'t', b'\g<1> not'),
        (b'(\w+)\'ve', b'\g<1> have'),
        (b'(\w+)\'s', b'\g<1> is'),
        (b'(\w+)\'re', b'\g<1> are'),
        (b'(\w+)\'d', b'\g<1> would')
    ]
patterns = [(re.compile(regex), repl) for (regex, repl) in cont_patterns]

def prepare_for_char_n_gram(text):
  clean = bytes(text.lower(), encoding="utf-8") #lower case
  clean = clean.replace(b"\n", b" ")   # spaces and tabs
  clean = clean.replace(b"\t", b" ")
  clean = clean.replace(b"\b", b" ")
  clean = clean.replace(b"\r", b" ")
  for (pattern, repl) in patterns:
      clean = re.sub(pattern, repl, clean)  # contractions?
  exclude = re.compile(b'[%s]' % re.escape(bytes(string.punctuation, encoding='utf-8')))
  clean = b" ".join([exclude.sub(b'', token) for token in clean.split()])  # punctuation
  clean = re.sub(b"\d+", b" ", clean)  #numbers
  clean = re.sub(b'\s+', b' ', clean)   # spaces
  clean = re.sub(b" ", b"# #", clean)   # add # signs?
  clean = re.sub(b'\s+$', b'', clean) # ending spaces
  return str(clean, 'utf-8')
   
def count_regexp_occ(regexp="", text=None):
    return len(re.findall(regexp, text))

def perform_nlp(df):     # Check all sorts of content
    df["ant_slash_n"] = df["comment_text"].apply(lambda x: count_regexp_occ(r"\n", x))
    df["raw_word_len"] = df["comment_text"].apply(lambda x: len(x.split()))
    df["raw_char_len"] = df["comment_text"].apply(lambda x: len(x))
    df["nb_upper"] = df["comment_text"].apply(lambda x: count_regexp_occ(r"[A-Z]", x))
    df["nb_fk"] = df["comment_text"].apply(lambda x: count_regexp_occ(r"[Ff]\S{2}[Kk]", x))
    df["nb_sk"] = df["comment_text"].apply(lambda x: count_regexp_occ(r"[Ss]\S{2}[Kk]", x))
    df["nb_dk"] = df["comment_text"].apply(lambda x: count_regexp_occ(r"[dD]ick", x))
    df["nb_you"] = df["comment_text"].apply(lambda x: count_regexp_occ(r"\W[Yy]ou\W", x))
    df["nb_mother"] = df["comment_text"].apply(lambda x: count_regexp_occ(r"\Wmother\W", x))
    df["nb_nigger"] = df["comment_text"].apply(lambda x: count_regexp_occ(r"\Wnigger\W", x))
    df["start_with_columns"] = df["comment_text"].apply(lambda x: count_regexp_occ(r"^\:+", x))
    df["has_timestamp"] = df["comment_text"].apply(lambda x: count_regexp_occ
        (r"\d{2}|:\d{2}", x))
    df["has_date_long"] = df["comment_text"].apply(lambda x: count_regexp_occ
    (r"\D\d{2}:\d{2},\d{1,2} \w+ \d{4}", x))
    df["has_date_short"] = df["comment_text"].apply(lambda x: count_regexp_occ
        (r"\D\d{1,2} \w+ \d{4}", x))
    df["has_http"] = df["comment_text"].apply(lambda x: count_regexp_occ
        (r"http[s]{0,1}://\S+", x))
    df["has_mail"] = df["comment_text"].apply(lambda x: count_regexp_occ(r"\S+\@\w+\.\w+", x))
    df["has_emphasize_equal"] = df["comment_text"].apply(lambda x: count_regexp_occ
        (r"\={2}.+\={2}", x))
    df["has_emphasize_quotes"] = df["comment_text"].apply(lambda x: count_regexp_occ
        (r"\"{4}\S+\"{4}", x))
    
    ip_regexp = r"""(([0-9]|[1-9][0-9]|1[0-9]{2}|2[0-4][0-9]|25[0-5])\.)
        {3}([0-9]|[1-9][0-9]|1[0-9]{2}|2[0-4][0-9]|25[0-5])$"""
    # Now clean comments
    df["clean_comment"] = df["comment_text"].apply(lambda x: prepare_for_char_n_gram(x))

    df["clean_chars"] = df["clean_comment"].apply(lambda x: len(set(x)))
    df["clean_chars_ratio"] = df["clean_comment"].apply(lambda x: len(set(x))) /    \
        df["clean_comment"].apply(lambda x: 1 + min(99, len(x)))

    # Get the exact length in words and characters
    df["clean_word_len"] = df["clean_comment"].apply(lambda x: len(x.split()))
    df["clean_char_len"] = df["clean_comment"].apply(lambda x: len(x))


def char_analyzer(text):
  tokens = text.split()
  return [token[i: i+3] for token in tokens for i in range(len(token) - 2)]

gc.enable()

print('reading data')
train = pd.read_csv('../input/train.csv').fillna(' ')
test = pd.read_csv('../input/test.csv').fillna(' ')
class_names = list(train)[-6:]



perform_nlp(train)
print('nlp train done')
perform_nlp(test)
print('nlp test done')

train_text = train['clean_comment']
test_text = test['clean_comment']
all_text = pd.concat([train_text, test_text])


print("Creating numerical features")
num_features = [f_ for f_ in train.columns
                if f_ not in ["comment_text", "clean_comment", "id", "remaining_chars", 
                    'has_ip_address'] + class_names]


skl = MinMaxScaler()

train_num_features =(skl.fit_transform(train[num_features]))   #csr_matrix
test_num_features = (skl.fit_transform(test[num_features]))

print("Tfidf on word")
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 2),
    max_features=50000)                 #20000
word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)
del word_vectorizer
gc.collect()


print("Tfidf on char n_gram")
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    tokenizer=char_analyzer,
    analyzer='word',
    ngram_range=(1, 1),
    max_features=80000)                    #50000
char_vectorizer.fit(all_text)
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)
    
print((train_char_features>0).sum(axis=1).max())


gc.collect()


print("Stacking matrices")
csr_trn = hstack(
    [train_char_features, 
    train_word_features, 
    train_num_features],
  format='csr') #.tocsr()

csr_sub = hstack([test_char_features, 
        test_word_features, 
        test_num_features], 
  format='csr')


# del train_text
# del test_text
# del train_word_features
# del test_word_features
# del train_char_features
# del test_char_features
# del train_num_features
# del test_num_features
# gc.collect()



reading data
nlp train done
nlp test done
Creating numerical features
Tfidf on word
Tfidf on char n_gram
1362
Stacking matrices


In [ ]:

submission = pd.DataFrame.from_dict({'id': test['id']})

drop_f = [f_ for f_ in train if f_ not in ["id"] + class_names]
train.drop(drop_f, axis=1, inplace=True)


print("Scoring LogisticRegression")

scores = []
folds = KFold(n_splits=5, shuffle=True, random_state=42)     # 5
lgb_round_dict = defaultdict(int)
trn_lgbset = lgb.Dataset(csr_trn, free_raw_data=False) 
params = {
        "objective": "binary",
        'metric': "auc",
        "boosting_type": "dart",
        "tree_learner": "feature",   # default
        "verbose": 0,
        "bagging_fraction": 0.9,
        "feature_fraction": 0.9,
        "learning_rate": 0.05,
        "min_data_in_leaf": 5,# 5
        "num_leaves": 31,
        "min_split_gain": 0,  # 0.05
        "reg_alpha": 0.1, #.1,
        "num_threads": 4,
        "is_unbalance": "true"   # default
        }
lgb_rounds = 2500    #####

for class_name in class_names:
    print("Class %s scores : " % class_name)
    class_pred = np.zeros(len(train))
    train_target = train[class_name]
    trn_lgbset.set_label(train_target.values)
    
    for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train, train_target)):
        
        watchlist = [trn_lgbset.subset(trn_idx), 
                     trn_lgbset.subset(val_idx)]
      
        model = lgb.train(
            params=params,
            train_set=watchlist[0],
            num_boost_round=lgb_rounds,
            valid_sets=watchlist,
            early_stopping_rounds=75
        )
        class_pred[val_idx] = model.predict(trn_lgbset.data[val_idx], 
            num_iteration=model.best_iteration)
        score = roc_auc_score(train_target.values[val_idx], class_pred[val_idx])
        lgb_round_dict[class_name] += model.best_iteration
        print("\t Fold %d : %.6f in %3d rounds" % (n_fold + 1, score, model.best_iteration))
     
    print("full score : %.6f" % roc_auc_score(train_target, class_pred))
    scores.append(roc_auc_score(train_target, class_pred))
    train[class_name + "_oof"] = class_pred

# Save OOF predictions
train[["id"] + class_names + [f + "_oof" for f in class_names]].to_csv("lvl0_lgbm_clean_oof.csv",
                                                            index=False)
   

Scoring LogisticRegression
Class toxic scores : 
[1]	training's auc: 0.867493	valid_1's auc: 0.864412
Training until validation scores don't improve for 75 rounds.
[2]	training's auc: 0.881718	valid_1's auc: 0.877966
[3]	training's auc: 0.885466	valid_1's auc: 0.882578
[4]	training's auc: 0.889345	valid_1's auc: 0.885839
[5]	training's auc: 0.892756	valid_1's auc: 0.889502
[6]	training's auc: 0.89631	valid_1's auc: 0.892913
[7]	training's auc: 0.901123	valid_1's auc: 0.897755
[8]	training's auc: 0.900976	valid_1's auc: 0.897831
[9]	training's auc: 0.901501	valid_1's auc: 0.897779
[10]	training's auc: 0.901873	valid_1's auc: 0.899009
[11]	training's auc: 0.903267	valid_1's auc: 0.900106
[12]	training's auc: 0.903272	valid_1's auc: 0.900224
[13]	training's auc: 0.904146	valid_1's auc: 0.900976
[14]	training's auc: 0.904896	valid_1's auc: 0.901759
[15]	training's auc: 0.905412	valid_1's auc: 0.902222
[16]	training's auc: 0.906386	valid_1's auc: 0.9032
[17]	training's auc: 0.908985	valid_1

[151]	training's auc: 0.952699	valid_1's auc: 0.942682
[152]	training's auc: 0.952659	valid_1's auc: 0.942649
[153]	training's auc: 0.953007	valid_1's auc: 0.943004
[154]	training's auc: 0.95297	valid_1's auc: 0.942976
[155]	training's auc: 0.953238	valid_1's auc: 0.943388
[156]	training's auc: 0.953155	valid_1's auc: 0.943345
[157]	training's auc: 0.953158	valid_1's auc: 0.943373
[158]	training's auc: 0.953048	valid_1's auc: 0.943281
[159]	training's auc: 0.952958	valid_1's auc: 0.943209
[160]	training's auc: 0.952873	valid_1's auc: 0.943141
[161]	training's auc: 0.952937	valid_1's auc: 0.943062
[162]	training's auc: 0.952946	valid_1's auc: 0.943059
[163]	training's auc: 0.953248	valid_1's auc: 0.943423
[164]	training's auc: 0.953555	valid_1's auc: 0.943694
[165]	training's auc: 0.953924	valid_1's auc: 0.943991
[166]	training's auc: 0.954286	valid_1's auc: 0.944254
[167]	training's auc: 0.954872	valid_1's auc: 0.94485
[168]	training's auc: 0.955244	valid_1's auc: 0.945245
[169]	traini

[301]	training's auc: 0.97122	valid_1's auc: 0.958345
[302]	training's auc: 0.97143	valid_1's auc: 0.958516
[303]	training's auc: 0.971383	valid_1's auc: 0.958485
[304]	training's auc: 0.971335	valid_1's auc: 0.958448
[305]	training's auc: 0.971277	valid_1's auc: 0.958388
[306]	training's auc: 0.971239	valid_1's auc: 0.958366
[307]	training's auc: 0.971243	valid_1's auc: 0.958366
[308]	training's auc: 0.971361	valid_1's auc: 0.958481
[309]	training's auc: 0.971344	valid_1's auc: 0.958469
[310]	training's auc: 0.971498	valid_1's auc: 0.958569
[311]	training's auc: 0.971669	valid_1's auc: 0.958709
[312]	training's auc: 0.971814	valid_1's auc: 0.958794
[313]	training's auc: 0.971981	valid_1's auc: 0.958942
[314]	training's auc: 0.971953	valid_1's auc: 0.958913
[315]	training's auc: 0.97191	valid_1's auc: 0.958877
[316]	training's auc: 0.972102	valid_1's auc: 0.959163
[317]	training's auc: 0.97225	valid_1's auc: 0.959268
[318]	training's auc: 0.972369	valid_1's auc: 0.959297
[319]	training

[451]	training's auc: 0.981321	valid_1's auc: 0.966773
[452]	training's auc: 0.981298	valid_1's auc: 0.96675
[453]	training's auc: 0.981277	valid_1's auc: 0.966731
[454]	training's auc: 0.981266	valid_1's auc: 0.966722
[455]	training's auc: 0.981368	valid_1's auc: 0.966764
[456]	training's auc: 0.981348	valid_1's auc: 0.966753
[457]	training's auc: 0.981309	valid_1's auc: 0.966713
[458]	training's auc: 0.981439	valid_1's auc: 0.966794
[459]	training's auc: 0.981542	valid_1's auc: 0.966912
[460]	training's auc: 0.981534	valid_1's auc: 0.966903
[461]	training's auc: 0.981635	valid_1's auc: 0.966962
[462]	training's auc: 0.981604	valid_1's auc: 0.966932
[463]	training's auc: 0.981576	valid_1's auc: 0.966912
[464]	training's auc: 0.98168	valid_1's auc: 0.967015
[465]	training's auc: 0.981783	valid_1's auc: 0.967031
[466]	training's auc: 0.981759	valid_1's auc: 0.967009
[467]	training's auc: 0.981848	valid_1's auc: 0.967098
[468]	training's auc: 0.981838	valid_1's auc: 0.967091
[469]	traini

[601]	training's auc: 0.985891	valid_1's auc: 0.970135
[602]	training's auc: 0.985885	valid_1's auc: 0.970127
[603]	training's auc: 0.985924	valid_1's auc: 0.970175
[604]	training's auc: 0.985916	valid_1's auc: 0.970166
[605]	training's auc: 0.985907	valid_1's auc: 0.970155
[606]	training's auc: 0.985899	valid_1's auc: 0.970148
[607]	training's auc: 0.985897	valid_1's auc: 0.970147
[608]	training's auc: 0.985962	valid_1's auc: 0.970196
[609]	training's auc: 0.98601	valid_1's auc: 0.970247
[610]	training's auc: 0.986032	valid_1's auc: 0.970345
[611]	training's auc: 0.986093	valid_1's auc: 0.970365
[612]	training's auc: 0.986159	valid_1's auc: 0.970406
[613]	training's auc: 0.986145	valid_1's auc: 0.970388
[614]	training's auc: 0.986194	valid_1's auc: 0.970397
[615]	training's auc: 0.986235	valid_1's auc: 0.970395
[616]	training's auc: 0.986276	valid_1's auc: 0.970374
[617]	training's auc: 0.986274	valid_1's auc: 0.970373
[618]	training's auc: 0.986318	valid_1's auc: 0.970371
[619]	train

...

[1]	training's auc: 0.915978	valid_1's auc: 0.906818
Training until validation scores don't improve for 75 rounds.
[2]	training's auc: 0.921783	valid_1's auc: 0.912369
[3]	training's auc: 0.92563	valid_1's auc: 0.914373
[4]	training's auc: 0.927538	valid_1's auc: 0.915702
[5]	training's auc: 0.932163	valid_1's auc: 0.921118
[6]	training's auc: 0.93734	valid_1's auc: 0.927061
[7]	training's auc: 0.937934	valid_1's auc: 0.927231
[8]	training's auc: 0.938469	valid_1's auc: 0.927664
[9]	training's auc: 0.939885	valid_1's auc: 0.928763
[10]	training's auc: 0.939816	valid_1's auc: 0.928483
[11]	training's auc: 0.941691	valid_1's auc: 0.930531
[12]	training's auc: 0.941805	valid_1's auc: 0.930102
[13]	training's auc: 0.942329	valid_1's auc: 0.930358
[14]	training's auc: 0.943391	valid_1's auc: 0.931584
[15]	training's auc: 0.944565	valid_1's auc: 0.932965
[16]	training's auc: 0.944901	valid_1's auc: 0.933409
[17]	training's auc: 0.945167	valid_1's auc: 0.934037
[18]	training's auc: 0.947355	v

[152]	training's auc: 0.975668	valid_1's auc: 0.961411
[153]	training's auc: 0.975773	valid_1's auc: 0.961459
[154]	training's auc: 0.975761	valid_1's auc: 0.961465
[155]	training's auc: 0.975941	valid_1's auc: 0.961661
[156]	training's auc: 0.975878	valid_1's auc: 0.961596
[157]	training's auc: 0.975858	valid_1's auc: 0.96158
[158]	training's auc: 0.975794	valid_1's auc: 0.961513
[159]	training's auc: 0.975708	valid_1's auc: 0.9614
[160]	training's auc: 0.975643	valid_1's auc: 0.961311
[161]	training's auc: 0.975815	valid_1's auc: 0.961559
[162]	training's auc: 0.975834	valid_1's auc: 0.961601
[163]	training's auc: 0.975896	valid_1's auc: 0.961818
[164]	training's auc: 0.975971	valid_1's auc: 0.961889
[165]	training's auc: 0.976106	valid_1's auc: 0.961978
[166]	training's auc: 0.976395	valid_1's auc: 0.962338
[167]	training's auc: 0.976557	valid_1's auc: 0.962482
[168]	training's auc: 0.976691	valid_1's auc: 0.962531
[169]	training's auc: 0.976918	valid_1's auc: 0.962779
[170]	trainin

[302]	training's auc: 0.986138	valid_1's auc: 0.971136
[303]	training's auc: 0.986112	valid_1's auc: 0.971126
[304]	training's auc: 0.986091	valid_1's auc: 0.971106
[305]	training's auc: 0.986063	valid_1's auc: 0.971077
[306]	training's auc: 0.986043	valid_1's auc: 0.97105
[307]	training's auc: 0.986045	valid_1's auc: 0.971045
[308]	training's auc: 0.986127	valid_1's auc: 0.971071
[309]	training's auc: 0.986115	valid_1's auc: 0.971058
[310]	training's auc: 0.986168	valid_1's auc: 0.971068
[311]	training's auc: 0.986226	valid_1's auc: 0.971079
[312]	training's auc: 0.986313	valid_1's auc: 0.971215
[313]	training's auc: 0.986394	valid_1's auc: 0.971248
[314]	training's auc: 0.98638	valid_1's auc: 0.971233
[315]	training's auc: 0.986356	valid_1's auc: 0.971214
[316]	training's auc: 0.986458	valid_1's auc: 0.971251
[317]	training's auc: 0.98652	valid_1's auc: 0.971369
[318]	training's auc: 0.98661	valid_1's auc: 0.971369
[319]	training's auc: 0.986592	valid_1's auc: 0.971342
[320]	training

[452]	training's auc: 0.990789	valid_1's auc: 0.975635
[453]	training's auc: 0.990778	valid_1's auc: 0.975619
[454]	training's auc: 0.990772	valid_1's auc: 0.975612
[455]	training's auc: 0.990809	valid_1's auc: 0.975654
[456]	training's auc: 0.990801	valid_1's auc: 0.975644
[457]	training's auc: 0.990782	valid_1's auc: 0.975623
[458]	training's auc: 0.99081	valid_1's auc: 0.975692
[459]	training's auc: 0.990862	valid_1's auc: 0.975706
[460]	training's auc: 0.990857	valid_1's auc: 0.975703
[461]	training's auc: 0.9909	valid_1's auc: 0.97576
[462]	training's auc: 0.990886	valid_1's auc: 0.975746
[463]	training's auc: 0.990874	valid_1's auc: 0.975727
[464]	training's auc: 0.990914	valid_1's auc: 0.975753
[465]	training's auc: 0.990948	valid_1's auc: 0.975778
[466]	training's auc: 0.990937	valid_1's auc: 0.975763
[467]	training's auc: 0.990975	valid_1's auc: 0.975826
[468]	training's auc: 0.990971	valid_1's auc: 0.975817
[469]	training's auc: 0.991001	valid_1's auc: 0.975804
[470]	training

[602]	training's auc: 0.992542	valid_1's auc: 0.977269
[603]	training's auc: 0.992558	valid_1's auc: 0.977331
[604]	training's auc: 0.992554	valid_1's auc: 0.977324
[605]	training's auc: 0.99255	valid_1's auc: 0.977313
[606]	training's auc: 0.992546	valid_1's auc: 0.977306
[607]	training's auc: 0.992544	valid_1's auc: 0.9773
[608]	training's auc: 0.99256	valid_1's auc: 0.9773
[609]	training's auc: 0.99257	valid_1's auc: 0.977311
[610]	training's auc: 0.992596	valid_1's auc: 0.97732
[611]	training's auc: 0.992636	valid_1's auc: 0.977382
[612]	training's auc: 0.992656	valid_1's auc: 0.977431
[613]	training's auc: 0.992647	valid_1's auc: 0.977421
[614]	training's auc: 0.992676	valid_1's auc: 0.977419
[615]	training's auc: 0.992731	valid_1's auc: 0.977436
[616]	training's auc: 0.99276	valid_1's auc: 0.977445
[617]	training's auc: 0.992757	valid_1's auc: 0.977437
[618]	training's auc: 0.992777	valid_1's auc: 0.977485
[619]	training's auc: 0.992796	valid_1's auc: 0.977535
[620]	training's au

In [ ]:
                                                         float_format="%.8f")

print('Total CV score is {}'.format(np.mean(scores)))


In [ ]:

print("Predicting probabilities")

for class_name in class_names:
    print("Predicting probabilities for {}".format(class_name))
    train_target = train[class_name]
    trn_lgbset.set_label(train_target.values)
    
    model = lgb.train(params=params,
                      train_set=trn_lgbset,
                      num_boost_round=int(lgb_round_dict[class_name] / folds.n_splits)
                 )
    print(num_boost_round, model.best_iteration)
    submission[class_name] = model.predict(csr_sub, num_iteration=model.best_iteration)
submission.to_csv("../subs/lvl0_lgbm_clean_sub.csv", index=False, float_format="%.8f")